In [1]:
import os
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import time
import pandas as pd
%matplotlib inline

In [2]:
gt_folder_path = '/home/ec2-user/SageMaker/data/footprint/9831_unet_rn34_train_images/gt_binary'
pred_folder_path = '/home/ec2-user/SageMaker/data/footprint/9831_unet_rn34_train_images/pred_result'

###  Loop over gt folder path images

In [22]:
lst_gt_imgs= [gt_img_file for gt_img_file in os.listdir(gt_folder_path) if os.path.isfile(os.path.join(gt_folder_path, gt_img_file))]
lst_gt_imgs[:5], len(lst_gt_imgs)

(['image_1000_gt.tif',
  'image_18414_gt.tif',
  'image_12464_gt.tif',
  'image_14761_gt.tif',
  'image_6500_gt.tif'],
 9831)

In [23]:
def get_image(path, is_bw_img=True):
    gray_img = cv.imread(path, cv.IMREAD_GRAYSCALE)
    if gray_img.shape != (512, 512):
        raise Exception(' ***** get_image() fails for image : ', path)
    if not is_bw_img:
        #if it is not bw image already then conver it to bw image.
        th, gray_img = cv.threshold(gray_img, 100, 1, cv.THRESH_BINARY)
        
    return gray_img

In [24]:
def find_iou(gt_bw_img, pred_bw_img):
    if(gt_bw_img.shape == pred_bw_img.shape):
        intersection = np.logical_and(gt_bw_img, pred_bw_img)
        union = np.logical_or(gt_bw_img, pred_bw_img)
        iou_score = np.sum(intersection) / np.sum(union)
        return iou_score
    else:
        raise Exception('***** find_iou() fails.')

In [25]:
def write_current_progress_status(log_text):
    with open('progress.txt', 'a') as logs:
        curr_time = time.localtime()
        str_time = str(curr_time.tm_mday) + '/' + str(curr_time.tm_mon) + '/' + str(curr_time.tm_year) + ' \t ' + str(curr_time.tm_hour) + ':' + str(curr_time.tm_min) + ':' + str(curr_time.tm_sec)
        logs.write('\n' + log_text + ' \t ' + str_time)

In [26]:
def write_log(*log_text):
    with open('logs.txt', 'a') as logs:
        curr_time = time.localtime()
        str_time = str(curr_time.tm_mday) + '/' + str(curr_time.tm_mon) + '/' + str(curr_time.tm_year) + ' \t ' + str(curr_time.tm_hour) + ':' + str(curr_time.tm_min) + ':' + str(curr_time.tm_sec)
        logs.write('\n' + ' '.join(log_text) + ' \t ' + str_time)

In [27]:
range_found = 'no range found'
def find_iou_range(iou_score):
    if 0 <= iou_score <= 20:
        range_found = ' <= 20%'
    elif 20 <= iou_score <= 30:
        range_found = '20+ to 30%'
    elif 30 <= iou_score <= 40:
        range_found = '30+ to 40%'
    elif 40 <= iou_score <= 50:
        range_found = '40+ to 50%'
    elif 50 <= iou_score <= 60:
        range_found = '50+ to 60%'
    elif 60 <= iou_score <= 70:
        range_found = '60+ to 70%'
    elif 70 <= iou_score <= 80:
        range_found = '70+ to 80%'
    elif 80 <= iou_score <= 90:
        range_found = '80+ to 90%'        
    elif 90 <= iou_score <= 95:
        range_found = '90+ to 95%'        
    elif 95 <= iou_score <= 100:
        range_found = '95+ to 100%'                

    return range_found

In [28]:
lst_final_csv_output= []
for idx, img_name in enumerate(lst_gt_imgs, start=1):
    try:
        #Get gt image. since it is already b&w image no need to convert it to B&W image.
        gt_bw_img = get_image(os.path.join(gt_folder_path, img_name))

        #Get prediction image. since it is not already b&w image have to convert it to B&W image.
        pred_file_name = img_name.replace('_gt.tif', '_pred_binary.png')
        pred_bw_img = get_image(os.path.join(pred_folder_path, pred_file_name), False)
#         pred_bw_img = cv.copyMakeBorder(pred_bw_img, 10,10,10,10, cv.BORDER_REFLECT101)

        #find IOU score
        iou_score_original = find_iou(gt_bw_img, pred_bw_img)
        iou_score = '{0:.4f}'.format(iou_score_original)
        
        iou_perc = '{0:.4f}'.format(iou_score_original*100)
#         print(type(iou_perc), iou_perc)
        iou_range = find_iou_range(float(iou_perc))
        print_str = '# {} :\t {} \t {} \t {} \t {} \t {}'.format(idx, img_name, pred_file_name, iou_score, iou_perc, iou_range)
        print(print_str)
#         print('Processing Image # %d : %s \t %s \t %s \t %s' % (idx, img_name, pred_file_name, iou_score, iou_perc))
        write_current_progress_status(print_str)
        #construct LIST to store it in csv file
        lst_final_csv_output.append([idx, img_name, iou_score , iou_perc, iou_range])
    except Exception as exception:
        print('-' * 50)
        print('Exception raised %s for the Image %s' % (str(exception), img_name))
        print('-' * 50)
        write_log(str(exception), img_name )
    finally:
        print('~' * 100)
        
#store the csv output
df = pd.DataFrame(lst_final_csv_output, columns=['No', 'image_name', 'IOU_Score', 'IOU_Perc', 'Class'])
df.to_csv('iou_score.csv', index=False)
print('total files processed : ', idx)
print('csv generated successfully')

# 1 :	 image_1000_gt.tif 	 image_1000_pred_binary.png 	 0.9915 	 99.1516 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 2 :	 image_18414_gt.tif 	 image_18414_pred_binary.png 	 0.9655 	 96.5541 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 3 :	 image_12464_gt.tif 	 image_12464_pred_binary.png 	 0.9504 	 95.0444 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 4 :	 image_14761_gt.tif 	 image_14761_pred_binary.png 	 0.8632 	 86.3222 	 80+ to 90%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 5 :	 image_6500_gt.tif 	 image_6500_pred_binary.png 	 0.9884 	 98.8420 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 6 :	 image_6132_gt.tif 	 image_6132_pred_binary.png 	

# 50 :	 image_11046_gt.tif 	 image_11046_pred_binary.png 	 0.9474 	 94.7363 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 51 :	 image_5732_gt.tif 	 image_5732_pred_binary.png 	 0.8618 	 86.1847 	 80+ to 90%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 52 :	 image_14323_gt.tif 	 image_14323_pred_binary.png 	 0.9595 	 95.9526 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 53 :	 image_18554_gt.tif 	 image_18554_pred_binary.png 	 0.9021 	 90.2110 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 54 :	 image_4832_gt.tif 	 image_4832_pred_binary.png 	 0.9596 	 95.9571 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 55 :	 image_10129_gt.tif 	 image_10129_pred_binary

# 97 :	 image_9720_gt.tif 	 image_9720_pred_binary.png 	 0.9330 	 93.2955 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 98 :	 image_8969_gt.tif 	 image_8969_pred_binary.png 	 0.8309 	 83.0865 	 80+ to 90%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 99 :	 image_8340_gt.tif 	 image_8340_pred_binary.png 	 0.6383 	 63.8279 	 60+ to 70%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 100 :	 image_12586_gt.tif 	 image_12586_pred_binary.png 	 0.9052 	 90.5232 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 101 :	 image_14212_gt.tif 	 image_14212_pred_binary.png 	 0.9522 	 95.2237 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 102 :	 image_15824_gt.tif 	 image_15824_pred_binary

# 146 :	 image_12526_gt.tif 	 image_12526_pred_binary.png 	 0.8991 	 89.9124 	 80+ to 90%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 147 :	 image_8431_gt.tif 	 image_8431_pred_binary.png 	 0.0000 	 0.0000 	  <= 20%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 148 :	 image_9049_gt.tif 	 image_9049_pred_binary.png 	 0.9650 	 96.5044 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 149 :	 image_9430_gt.tif 	 image_9430_pred_binary.png 	 0.9560 	 95.5995 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 150 :	 image_10190_gt.tif 	 image_10190_pred_binary.png 	 0.9230 	 92.3018 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 151 :	 image_808_gt.tif 	 image_808_pred_binary.png

# 191 :	 image_7883_gt.tif 	 image_7883_pred_binary.png 	 0.9342 	 93.4181 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 192 :	 image_15238_gt.tif 	 image_15238_pred_binary.png 	 0.9184 	 91.8406 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 193 :	 image_13005_gt.tif 	 image_13005_pred_binary.png 	 0.9423 	 94.2335 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 194 :	 image_2348_gt.tif 	 image_2348_pred_binary.png 	 0.9769 	 97.6903 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 195 :	 image_1725_gt.tif 	 image_1725_pred_binary.png 	 0.9375 	 93.7466 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 196 :	 image_10747_gt.tif 	 image_10747_pred_bin

...

# 9628 :	 image_13474_gt.tif 	 image_13474_pred_binary.png 	 0.9504 	 95.0415 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 9629 :	 image_18012_gt.tif 	 image_18012_pred_binary.png 	 0.9561 	 95.6144 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 9630 :	 image_3140_gt.tif 	 image_3140_pred_binary.png 	 0.9423 	 94.2291 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 9631 :	 image_8290_gt.tif 	 image_8290_pred_binary.png 	 0.9802 	 98.0175 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 9632 :	 image_11699_gt.tif 	 image_11699_pred_binary.png 	 0.9474 	 94.7416 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 9633 :	 image_8380_gt.tif 	 image_8380_

# 9680 :	 image_11600_gt.tif 	 image_11600_pred_binary.png 	 0.9225 	 92.2540 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 9681 :	 image_11230_gt.tif 	 image_11230_pred_binary.png 	 0.9604 	 96.0384 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 9682 :	 image_8047_gt.tif 	 image_8047_pred_binary.png 	 0.5848 	 58.4826 	 50+ to 60%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 9683 :	 image_5299_gt.tif 	 image_5299_pred_binary.png 	 0.9729 	 97.2863 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 9684 :	 image_17013_gt.tif 	 image_17013_pred_binary.png 	 0.9203 	 92.0258 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 9685 :	 image_6454_gt.tif 	 image_6454_p

# 9731 :	 image_6455_gt.tif 	 image_6455_pred_binary.png 	 0.8122 	 81.2246 	 80+ to 90%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 9732 :	 image_11851_gt.tif 	 image_11851_pred_binary.png 	 0.9348 	 93.4824 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 9733 :	 image_5427_gt.tif 	 image_5427_pred_binary.png 	 0.8120 	 81.2009 	 80+ to 90%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 9734 :	 image_513_gt.tif 	 image_513_pred_binary.png 	 0.8948 	 89.4804 	 80+ to 90%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 9735 :	 image_4077_gt.tif 	 image_4077_pred_binary.png 	 0.9426 	 94.2582 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 9736 :	 image_11403_gt.tif 	 image_11403_pred_bi

# 9777 :	 image_6756_gt.tif 	 image_6756_pred_binary.png 	 0.9578 	 95.7827 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 9778 :	 image_7989_gt.tif 	 image_7989_pred_binary.png 	 0.5464 	 54.6371 	 50+ to 60%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 9779 :	 image_13581_gt.tif 	 image_13581_pred_binary.png 	 0.7928 	 79.2752 	 70+ to 80%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 9780 :	 image_12867_gt.tif 	 image_12867_pred_binary.png 	 0.6454 	 64.5383 	 60+ to 70%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 9781 :	 image_11658_gt.tif 	 image_11658_pred_binary.png 	 0.9312 	 93.1151 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 9782 :	 image_18435_gt.tif 	 image_18435_

# 9824 :	 image_12724_gt.tif 	 image_12724_pred_binary.png 	 0.8947 	 89.4693 	 80+ to 90%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 9825 :	 image_6042_gt.tif 	 image_6042_pred_binary.png 	 0.7497 	 74.9735 	 70+ to 80%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 9826 :	 image_11704_gt.tif 	 image_11704_pred_binary.png 	 0.7953 	 79.5306 	 70+ to 80%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 9827 :	 image_14922_gt.tif 	 image_14922_pred_binary.png 	 0.9335 	 93.3495 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 9828 :	 image_17861_gt.tif 	 image_17861_pred_binary.png 	 0.8014 	 80.1387 	 80+ to 90%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# 9829 :	 image_5016_gt.tif 	 image_5016_p